In [1]:
# One-liner to start the debugger here.
# from IPython.core.debugger import Tracer; Tracer()()
# np.set_printoptions(threshold=np.inf)

In [2]:
import numpy as np
import pdb
from sklearn.metrics import pairwise
import scipy.sparse as sp
import matplotlib.pyplot as plt
import random
import time
#import math
import matplotlib.pyplot as plt
#from hashlib import sha1

In [3]:
def chunk_generator(fname, chunk_size=1000):
        data = []
        counter = 0

        with open(fname, 'r') as ifile:

            for line in ifile:
                if counter < chunk_size:
                    data.append([int(p) for p in line.split(" ")])
                    counter += 1
                if counter == chunk_size:
                    counter = 0
                    yield data
                    data = []
                    
            # process remaining elements
            if len(data) > 0:
                yield data

class ShingleFileParser:
    def __init__(self, fname):
        cindex = 0;
        for chunk in chunk_generator(fname, 40000):
            if cindex == 0 :
                self.D = chunk[0][0]
                self.W = chunk[1][0]
                self.X = sp.lil_matrix ((self.D,self.W))
                for i in range(3,len(chunk)):
                    self.add_to_matrix(chunk[i])
            else:
                for line in chunk :
                    self.add_to_matrix(line)
            cindex = cindex + 1
        
    def add_to_matrix(self,item):
            D_i = item[0] - 1
            W_i = item[1] - 1
            self.X[D_i,W_i] = 1
    
    def save_csc(self,filename):
        print("converting to csc (column-access optimized) sparse format ...")
        self.X = sp.csc_matrix(self.X)
        print("saving matrix with " + str(len(self.X.data)) + " elements")
        np.savez(filename,data = self.X.data ,indices=self.X.indices,
                 indptr = self.X.indptr, shape = self.X.shape)
        print ("content saved to " + filename)


In [7]:
start = time.time()
sf_parser =  ShingleFileParser('data/docword.enron.txt')
end = time.time()
print("Parsing the file took " + str(end - start) + " seconds")
sf_parser.save_csc('csc_kos');
del sf_parser

Parsing the file took 18.0582771301 seconds
converting to csc (column-access optimized) sparse format ...
saving matrix with 3710420 elements
content saved to csc_kos


In [4]:
def load_sparse_csc(filename):
    loader = np.load(filename)
    return sp.csc_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])
D = load_sparse_csc('csc_kos.npz')
D = sp.csr_matrix(D)

Q = D[0:100,:]
P = D[100:,:]
print("Point set shape: %s" % str(P.shape))
print("Query set shape: %s" % str(Q.shape))

Point set shape: (39761, 28102)
Query set shape: (100, 28102)


In [31]:
t0 = time.time()

J = Q.dot(P.T).todense()

log_freq=5000
for q in range(Q.shape[0]):
    for p in range(P.shape[0]):
        # Compute the set size
        if p%log_freq is 0: print("Similarity for row query point %i/%i with point %i/%i" %(q+1,Q.shape[0],p+1,P.shape[0]) )
        set1 = P.getrow(p).getnnz()
        if p%log_freq is 0: print("nonzeros in P(%i)"%set1)
        set2 = Q.getrow(q).getnnz()
        if p%log_freq is 0: print("nonzeros in Q(%i)"%set2)
        if p%log_freq is 0: print("union count (%i)"% J[q,p])
        # Compute Jaccard
        J[q, p] = J[q, p] / (set1 + set2 - J[q, p])
        if p%log_freq is 0: print("Jaccard (%f), elapsed "% J[q,p])

#    print(i)

# Calculate the elapsed time (in seconds)
elapsed = time.time() - t0
print ("Bruteforce computation took ", elapsed, "sec")
np.savetxt("Bruteforce.txt", J, delimiter = '\t')

print("Average Jaccard similarity: ", np.sum(J) / (Q.shape[0] * P.shape[0]))

Similarity for row query point 1/100 with point 1/39761
nonzeros in P(34)
nonzeros in Q(30)
union count (1)
Jaccard (0.015873)


/usr/lib/python2.7/dist-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Similarity for row query point 1/100 with point 5001/39761
nonzeros in P(19)
nonzeros in Q(30)
union count (1)
Jaccard (0.020833)
Similarity for row query point 1/100 with point 10001/39761
nonzeros in P(46)
nonzeros in Q(30)
union count (1)
Jaccard (0.013333)
Similarity for row query point 1/100 with point 15001/39761
nonzeros in P(271)
nonzeros in Q(30)
union count (0)
Jaccard (0.000000)
Similarity for row query point 1/100 with point 20001/39761
nonzeros in P(238)
nonzeros in Q(30)
union count (3)
Jaccard (0.011321)
Similarity for row query point 1/100 with point 25001/39761
nonzeros in P(27)
nonzeros in Q(30)
union count (0)
Jaccard (0.000000)
Similarity for row query point 1/100 with point 30001/39761
nonzeros in P(174)
nonzeros in Q(30)
union count (2)
Jaccard (0.009901)
Similarity for row query point 1/100 with point 35001/39761
nonzeros in P(33)
nonzeros in Q(30)
union count (1)
Jaccard (0.016129)
Similarity for row query point 2/100 with point 1/39761
nonzeros in P(34)
nonzero

IndexError: tuple index out of range

In [5]:
bruteforce = np.loadtxt("Bruteforce.txt")
sim80 = np.where(bruteforce>=0.8)
sim80 = np.vstack([sim80[0],sim80[1]])
for i in range(sim80.shape[1]):
    print("Similar value number %i :J(%i,%i) = %f" % (i+1,sim80[0,i],sim80[1,i],bruteforce[sim80[0,i],sim80[1,i]]))

Similar value number 1 :J(0,733) = 1.000000
Similar value number 2 :J(1,173) = 0.910112
Similar value number 3 :J(3,269) = 0.952381
Similar value number 4 :J(9,140) = 1.000000
Similar value number 5 :J(9,512) = 1.000000
Similar value number 6 :J(10,765) = 0.952381
Similar value number 7 :J(11,87) = 1.000000
Similar value number 8 :J(12,67) = 1.000000
Similar value number 9 :J(19,761) = 1.000000
Similar value number 10 :J(23,199) = 1.000000
Similar value number 11 :J(27,216) = 1.000000
Similar value number 12 :J(29,138) = 1.000000
Similar value number 13 :J(33,666) = 1.000000
Similar value number 14 :J(36,224) = 0.857143
Similar value number 15 :J(39,23530) = 1.000000
Similar value number 16 :J(39,23715) = 0.889796
Similar value number 17 :J(42,694) = 1.000000
Similar value number 18 :J(43,221) = 1.000000
Similar value number 19 :J(43,248) = 1.000000
Similar value number 20 :J(48,211) = 1.000000
Similar value number 21 :J(53,697) = 1.000000
Similar value number 22 :J(59,624) = 1.000000


In [6]:
def getPrime(): return 9973
#------------------------------------------------------------------------------
# Generate a list of 'k' random coefficients for the random hash functions,
# while ensuring that the same value does not appear multiple times in the 
# list.
def pickRandomCoeffs(k):
    
    # Generate random integers in range {0, ..., maxInt}
    maxInt = 2**16-1
    
    # Create a list of 'k' random values.
    randList = []
  
    while k > 0:
        # Get a random shingle ID.
        randIndex = random.randint(0, maxInt) 
  
        # Ensure that each random number is unique.
        while randIndex in randList:
            randIndex = random.randint(0, maxInt) 
    
        # Add the random number to the list.
        randList.append(randIndex)
        k = k - 1
    
    return randList

def generateMinHash(k,X):

    N=X.shape[1]
        
    maxInt = 2**16 - 1

    # Our random hash function will take the form of:
    #   h(x) = (a*x + b) % c
    # Where 'x' is the input value, 'a' and 'b' are random coefficients, and 'c' is
    # a large prime number 
    
    coeffA = pickRandomCoeffs(k)
    coeffB = pickRandomCoeffs(k)
        
    minHash = np.zeros((N, k), dtype=np.int)
    for i in range(N):
             
        # Get doc in sparse format
        doc = X.getrow(i)
      
        # Get index and value of nonzero entries
        # row_idx is always 0 since the matrix has only one row
        # col_idx is the column indices
        # val is the value
        row_idx, col_idx, val = sp.find(doc)
      
        # For each of the random hash functions...
        for j in range(k):
          
        
            # Track the lowest hash ID seen. Initialize 'minHashCode' to be greater than
            # the maximum possible value output by the hash.
            minHashCode = maxInt + 1
        
            # For each shingle in the document...
            for idx in col_idx:
         
                # Evaluate the hash function.
                hashCode = (coeffA[j] * idx + coeffB[j]) % getPrime()
          
                # Track the lowest hash code seen.
                if hashCode < minHashCode:
                    minHashCode = hashCode
    
            # Add the smallest hash code value as component number 'idx' of the signature.
            minHash[i][j] = minHashCode
        
    return minHash

In [8]:
for r in range (3,10):
    for b in range (3,20):
        if((1-0.8**r)**b < 0.1):
            if(1-(1-0.4**r)**b<0.01):
                print("r:",r,"b:",b)
                
r=8
b=13
K=r*b
t0 = time.time()
MinHash = generateMinHash(K,D)

# Calculate the elapsed time (in seconds)
elapsed = time.time() - t0
print ("Generating MinHash signatures took ", elapsed, "sec")

('r:', 8, 'b:', 13)
('r:', 8, 'b:', 14)
('r:', 8, 'b:', 15)
('r:', 9, 'b:', 16)
('r:', 9, 'b:', 17)
('r:', 9, 'b:', 18)
('r:', 9, 'b:', 19)
('Generating MinHash signatures took ', 303.0851049423218, 'sec')


In [9]:
# Generate random integers to compute hash value for LSH
random_list = np.random.random_integers(0, getPrime(), r)

# Create B empty tables
M = 20000
TABLEs = np.empty( (b, M), dtype=object)   
N = D.shape[1]
# For each table
for i in range(b):
    # Compute the index to get MinHash values
    idx1 = r * i
    idx2 = idx1 + r
    # For each document, hash it into the table
    for j in range(N):
        # Get a string from all values in the band
        minHashValues = MinHash[j][idx1 : idx2]
        # The hash value is simply dot product and modulo with N since each table has N buckets
        hashValue = np.dot(minHashValues, random_list) % M
        # If bucket is empty, creat a set() and insert document ID into set
        if not TABLEs[i][hashValue]:    
            # bucket is a set
            TABLEs[i][hashValue] = list()
            TABLEs[i][hashValue].append(j)
        else:    
            # Insert document ID into the initialized bucket
            TABLEs[i][hashValue].append(j) # insert into bucket

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: DeprecationWarning: This function is deprecated. Please call randint(0, 9973 + 1) instead
  from ipykernel import kernelapp as app


In [10]:

# similar pair threshold
J1 = 0.8
# far away pair threshold
J2 = 0.4

# Histogram to identify similar pair using MinHash
minHash_TruePair = sp.csr_matrix((N, N), dtype=np.int) # can be replaced by boolean
# Histogram to identify far away pair using MinHash
minHash_FarAwayPair = sp.csr_matrix((N, N), dtype=np.int) # can be replaced by boolean
# Histogram to identify far away pair using MinHash
minHash_AllPair = sp.csr_matrix((N, N), dtype=np.int) # can be replaced by boolean

# For each table
for i in range(b):
    # For each bucket
    for j in range(M):
        # If the table has some entries
        if TABLEs[i][j] is not None:
            candidate = TABLEs[i][j]
            bucket_size = len(candidate)
            # There should be more than 1 document in a bucket
            if bucket_size > 1:                
                # Loop all pairs in a bucket (a naive approach)
                # We can do better since candidate is a sorted list
                # and each pair is counted twice: (d1, d2) and (d2, d1) 
                for k in range(bucket_size):
                    for l in range(bucket_size):
                        idx1, idx2 = candidate[k],candidate[l]
                        # don't compare elements of the same set
                        if l > k and idx2 >= 100 and idx1<100:
                            # Get the document ID                              
                            # Note that any pair (d1, d2) might collide on several hash tables
                            minHash_AllPair[idx1,idx2] = 1
                            minHash_AllPair[idx2,idx1] = 1
                            
                            # Get true jaccard similarity
                            sim = bruteforce[idx1][idx2-100]
                            
                            # Verify
                            if sim >= J1:
                                minHash_TruePair[idx1,idx2] = 1
                                minHash_TruePair[idx2,idx1] = 1
                                
                            if sim <= J2:
                                minHash_FarAwayPair[idx1,idx2] = 1
                                minHash_FarAwayPair[idx2,idx1] = 1

/usr/lib/python2.7/dist-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [11]:
                            
print("Candidate size: ", minHash_AllPair.getnnz() / 2) # dont count each pair twice (d1, d2) = (d2, d1)
print("Number of true pair in the candidate set: ", minHash_TruePair.getnnz() / 2) # dont count each pair twice (d1, d2) = (d2, d1)

# Number of True Pairs J >= 0.8 without considering identical pairs, e.g., (d1, d1)
numTruePair = sim80.shape[1]
print("Number of true pairs in bruteforce: ", numTruePair )

# False negative
falseNegative = np.count_nonzero(minHash_TruePair.getnnz())*1.0 / numTruePair
print("False negatives: ", 1 - falseNegative)

# False positive
falsePositive = (minHash_AllPair.getnnz() - minHash_TruePair.getnnz())*1.0/ (minHash_AllPair.getnnz())

print("False positives: ", falsePositive)    

# Number of Far Away Pairs J <= 0.4
numFarAwayPair = np.flatnonzero(bruteforce <= J2).size
colProb_FarAway = minHash_FarAwayPair.getnnz() *1.0/ numFarAwayPair

print("Probability that a far away pair is in the candidate set: ", colProb_FarAway)  

('Candidate size: ', 1888)
('Number of true pair in the candidate set: ', 43)
('Number of true pairs in bruteforce: ', 46)
('False negatives: ', 0.9782608695652174)
('False positives: ', 0.9772245762711864)
('Probability that a far away pair is in the candidate set: ', 0.000926084521183869)
